In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import MinMaxScaler
import os
from numpy.fft import fft 

# Функция для создания окон и применения FFT
def create_windows(data, window_size):
    X = []
    y = []
    for i in range(0, len(data) - window_size + 1, window_size):
        window = data[i:i + window_size]
        if window['class'].nunique() == 1:  # Проверяем, что в окне данные одного класса
            # Применяем FFT к каждому признаку в окне
            window_fft = np.abs(fft(window[features].values, axis=0))
            X.append(window_fft)
            y.append(window['class'].iloc[0])
    return np.array(X), np.array(y)

# Функция для низкочастотной фильтрации
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Загрузка данных из нескольких файлов
file_paths = [os.path.join(r"..\preprocessed_data", path) for path in os.listdir(r"..\preprocessed_data")]
data_list = []
for file_path in file_paths:
    temp_data = pd.read_csv(file_path)
    data_list.append(temp_data)

data = pd.concat(data_list, ignore_index=True)

features = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

window_size_seconds = 2
sampling_rate = 0.2
window_size_samples = int(window_size_seconds / sampling_rate)
X_windowed, y_windowed = create_windows(data, window_size_samples)

# Кодирование классов и разделение данных
unique_classes = np.unique(y_windowed)
class_mapping = {label: idx for idx, label in enumerate(unique_classes)}
y_windowed_encoded = np.vectorize(class_mapping.get)(y_windowed)
y_windowed_categorical = to_categorical(y_windowed_encoded)
X_train, X_test, y_train, y_test = train_test_split(
    X_windowed, y_windowed_categorical, test_size=0.2, random_state=42)


In [ ]:
X_windowed

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, InputLayer, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

model = Sequential()

# Добавление слоев
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
# inputs = Input(shape=(X_train.shape[1], X_train.shape[2], 1))

# 
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Conv2D(16, (1, 1), padding='valid')(x)  # Используем 1x1 свёртку для сохранения размерности
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(32, (2, 2), padding='valid')(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Conv2D(32, (1, 1), padding='valid')(x)  # Используем 1x1 свёртку для сохранения размерности
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# x = Flatten()(x)
# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

x = LSTM(128, return_sequences=True) (inputs)
x = LSTM(128) (x)
x = Dropout(0.5)(x)
x = LSTM(64) (x)
x = Dropout(0.5)(x)
prediction = Dense(y_train.shape[1], activation='softmax')(x)
model = Model(inputs=inputs, outputs=prediction)


# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:
from tensorflow.keras.utils import plot_model, model_to_dot

dot_img_file = 'model_1.png'


plot_model(model, to_file=dot_img_file, rankdir="LR",)

In [ ]:
table=pd.DataFrame(columns=["Type","Shape"])
for layer in model.layers:
    table = table.append({"Type": layer.__class__.__name__,"Shape":layer.output_shape}, ignore_index=True)

In [ ]:
table.head(100)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Оценка модели на тестовых данных
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Тестовая потеря: {test_loss}, Тестовая точность: {test_acc}")

# Графики потерь и точности
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Обучение')
plt.plot(history.history['val_loss'], label='Валидация')
plt.title('График потерь')
plt.xlabel('Эпоха')
plt.ylabel('Потери')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Обучение')
plt.plot(history.history['val_accuracy'], label='Валидация')
plt.title('График точности')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()
plt.show()

# Предсказания и матрица ошибок
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Предсказанные классы')
plt.ylabel('Истинные классы')
plt.title('Матрица ошибок')
plt.show()

# Отчет о классификации
print(classification_report(y_true_classes, y_pred_classes, target_names=unique_classes))
